<a href="https://colab.research.google.com/github/SeaOfFrost/BookCoverClassifier/blob/master/ResNet_Cover_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Required Folder and check for GPU

In [ ]:
ls

 ������ C �еľ�û�б�ǩ��
 �������к��� 8C05-667A

 c:\Users\a6377\Desktop\Library Inventory Management System SMU\classification ��Ŀ¼

2024/11/06  21:54    <DIR>          .
2024/11/06  21:44    <DIR>          ..
2024/11/06  20:55    <DIR>          CNN
2024/11/04  22:48    <DIR>          data
2024/11/04  23:10               813 main.py
2024/11/06  21:54            19,505 ResNet_Cover_Classifier.ipynb
2024/11/04  22:34       102,967,424 resnet50_weights_tf_dim_ordering_tf_kernels.h5
               3 ���ļ�    102,987,742 �ֽ�
               4 ��Ŀ¼ 430,339,002,368 �����ֽ�


In [2]:
cd drive/My Drive/Book Classifier/book-dataset/Task1/

[WinError 3] 系统找不到指定的路径。: 'drive/My Drive/Book Classifier/book-dataset/Task1/'
c:\Users\a6377\Desktop\Library Inventory Management System SMU\classification


C:\Users\a6377\AppData\Roaming\Python\Python312\site-packages\IPython\core\magics\osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [3]:
!nvidia-smi

'nvidia-smi' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


# Import Dependencies

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
plt.ion()   # interactive mode

# Load Data

In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = './images'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val', 'test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32,
                                             shuffle=True, num_workers=8)
              for x in ['train', 'val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

FileNotFoundError: [WinError 3] 系统找不到指定的路径。: './images\\train'

# Train Model

In [ ]:
def train_model(model, criterion, optimizer, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
#             if phase == 'train':
#                 scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model = models.resnet50(pretrained=True)
for param in model.parameters():
  param.requires_grad = False
model.fc = nn.Sequential(nn.Linear(2048, 512),
                         nn.ReLU(),
                         nn.Dropout(0.2),
                         nn.Linear(512, 10),
                         nn.LogSoftmax(dim=1))
model = model.to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

Train model and save weights after completion

In [ ]:
model_ft = train_model(model, criterion, optimizer, num_epochs=25)
torch.save(model_ft.state_dict(), 'Weights.pt')

Epoch 1/25
----------
train Loss: 1.7561 Acc: 0.3805
val Loss: 1.6226 Acc: 0.4417

Epoch 2/25
----------
train Loss: 1.7384 Acc: 0.3860
val Loss: 1.6124 Acc: 0.4393

Epoch 3/25
----------
train Loss: 1.7187 Acc: 0.4016
val Loss: 1.6007 Acc: 0.4446

Epoch 4/25
----------
train Loss: 1.7047 Acc: 0.4032
val Loss: 1.5810 Acc: 0.4405

Epoch 5/25
----------
train Loss: 1.6838 Acc: 0.4151
val Loss: 1.5724 Acc: 0.4498

Epoch 6/25
----------
train Loss: 1.6935 Acc: 0.4114
val Loss: 1.5883 Acc: 0.4457

Epoch 7/25
----------
train Loss: 1.6757 Acc: 0.4179
val Loss: 1.5650 Acc: 0.4527

Epoch 8/25
----------
train Loss: 1.6599 Acc: 0.4233
val Loss: 1.5588 Acc: 0.4673

Epoch 9/25
----------
train Loss: 1.6613 Acc: 0.4227
val Loss: 1.5589 Acc: 0.4562

Epoch 10/25
----------
train Loss: 1.6608 Acc: 0.4144
val Loss: 1.5673 Acc: 0.4492

Epoch 11/25
----------
train Loss: 1.6425 Acc: 0.4218
val Loss: 1.5467 Acc: 0.4446

Epoch 12/25
----------
train Loss: 1.6528 Acc: 0.4269
val Loss: 1.5587 Acc: 0.4516

E

# Test Model

In [ ]:
def test_model(model, criterion):
    
    path = 'Weights.pt'
    since = time.time()
    model.load_state_dict(torch.load(path))
    model = model.to(device)
    
    best_acc = 0.0
    model.eval()   # Set model to evaluate mode

    running_loss = 0.0
    running_corrects = 0
    total_preds = []
    total_labels = []

    # Iterate over data.
    for inputs, labels in dataloaders['test']:
        total_labels.append(labels)
        inputs = inputs.to(device)
        labels = labels.to(device)

        # forward
        # track history if only in train
        with torch.no_grad():
            outputs = model(inputs)
            # print(outputs.shape)
            total_preds.append(outputs.detach().cpu())
            _, preds = torch.max(outputs, 1)
            loss = criterion(outputs, labels)
        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    loss = running_loss / dataset_sizes['test']
    acc = running_corrects.double() / dataset_sizes['test']

    print('Loss: {:.4f} Acc: {:.4f}'.format(loss, acc))

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    
    return total_preds, total_labels

In [ ]:
preds, labels = test_model(model, criterion)

Loss: 1.6499 Acc: 0.4142
Training complete in 2m 7s


In [ ]:
from torch import topk

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

In [ ]:
total_top1 = 0
total_top3 = 0
total_top5 = 0
for i in range(len(preds)):
  top1, top3, top5 = accuracy(preds[i], labels[i], topk = (1, 3, 5))
  total_top1 += top1
  total_top3 += top3
  total_top5 += top5
print(f"Top 1 Accuracy is: {total_top1/len(preds)}")
print(f"Top 3 Accuracy is: {total_top3/len(preds)}")
print(f"Top 5 Accuracy is: {total_top5/len(preds)}")

Top 1 Accuracy is: 41.42361068725586
Top 3 Accuracy is: 71.61458587646484
Top 5 Accuracy is: 87.1875
